In [1]:
import pandas as pd
import sqlite3
import os
import re


conn = sqlite3.connect('test.db')
symbolDb = pd.read_excel("FinalList.xlsx", engine='openpyxl')
c = conn.cursor()

In [2]:
print(pd.read_sql("select * from SymbolName", conn))
print(pd.read_sql("select * from TradeCall", conn))
print(pd.read_sql("select * from SwingTable", conn))



     Coin                symbol_id    SymbolDb
0    ALGO   BINANCE_SPOT_ALGO_USDT        None
1   SUPER  BINANCE_SPOT_SUPER_USDT   SUPERUSDT
2   ALICE  BINANCE_SPOT_ALICE_USDT   ALICEBUSD
3     CRO     KUCOIN_SPOT_CRO_USDT     CROUSDT
4     KCS     KUCOIN_SPOT_KCS_USDT     KCSUSDT
5    POLS   BINANCE_SPOT_POLS_USDT    POLSUSDT
6     FTM    BINANCE_SPOT_FTM_USDT     FTMUSDT
7    MNGO    GATEIO_SPOT_MNGO_USDT        None
8     EQZ     KUCOIN_SPOT_EQZ_USDT     EQZUSDT
9    STEP    GATEIO_SPOT_STEP_USDT     STEPUSD
10    MLN    BINANCE_SPOT_MLN_USDT     MLNUSDT
11   LOCG    KUCOIN_SPOT_LOCG_USDT    LOCGUSDT
12    RAY    BINANCE_SPOT_RAY_USDT     RAYBUSD
13   MANA   BINANCE_SPOT_MANA_USDT    MANAUSDT
14    BNB    BINANCE_SPOT_BNB_USDT     BNBUSDT
15   DATA   BINANCE_SPOT_DATA_USDT    DATABUSD
16    ADA    BINANCE_SPOT_ADA_USDT     ADAUSDT
17    LRC    BINANCE_SPOT_LRC_USDT        None
18    LTC    BINANCE_SPOT_LTC_USDT     LTCUSDT
19   HBAR   BINANCE_SPOT_HBAR_USDT    HBARUSDT
20    PHA    

In [2]:
df = pd.read_excel('FinalList.xlsx', engine='openpyxl')

df.loc[df['symbol_id'] == 'BINANCE_SPOT_SUPER_USDT', 'SymbolDb'].values[0]

' SUPERUSDT'

In [3]:
def getSymbolIds():
    df = pd.read_excel('FinalList.xlsx', engine='openpyxl')
    symbol_list = df['symbol_id']
    return symbol_list.tolist()

In [14]:
symbols = getSymbolIds()
lis = []
for symbol in symbols:
    for root, subdirectories, files in os.walk('swings/'):
                for subdirectory in subdirectories:
                    folderName = os.path.join(root, subdirectory)
                    df = pd.read_excel(folderName + '/' +
                                       symbol + '.xlsx', engine='openpyxl')
                    lis.append(df)
SwingDf = pd.concat(lis)
SwingDf = SwingDf.iloc[: , 2:]
SwingDf.to_sql('SwingTable', conn, if_exists='append', index=False)

              

In [4]:
newDf = pd.read_sql("""    SELECT
    T1.*,
    DerivedTable.swings
FROM TradeCall T1
JOIN
(
    SELECT 
        T2.*,
        ROW_NUMBER() OVER (PARTITION BY T2.SymbolDb ORDER BY T2.time_close DESC) AS RowNum
    FROM SwingTable T2, TradeCall T1
    WHERE datetime(T2.time_close) < datetime(T1.TimeStamp)
    AND T2.swings IS NOT NULL AND 
    CASE 
        WHEN T1. 
        THEN co.DTEntered
        ELSE '2011-01-01' 
                     END 
) DerivedTable
ON T1.Ticker = DerivedTable.SymbolDb
WHERE DerivedTable.RowNum = 1""", conn)
newDf.to_excel("CalculatedSwing.xlsx", index = False)
print(newDf)

     Ticker Position            TimeStamp        swings
0   STEPUSD     LONG  2021-11-22 09:00:08 -1.676000e-02
1  SHIBUSDT     LONG  2021-11-22 06:42:13  5.600000e-07


In [2]:
conn.execute("delete from  TradeCall;")


In [4]:
conn.execute("ALTER TABLE SwingTable ADD swing_price TEXT;")
